In [28]:
import eikon as ek
import pandas as pd
import numpy as np
import datetime
ek.set_app_key("69de0ce7c5e2468e90a6058ce79f6558ebe3ee30")

'https://community.developers.refinitiv.com/questions/73493/get-eps-historical-data-for-stocks.html

# Economic Story

**TR.RevenueActValue.date** - prints the datetime

## Part 1: Accuracy

**TR.EPSActValue** - The company's actual value normalized to reflect the I/B/E/S default currency and corporate actions (e.g. stock splits). Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.

**TR.EPSMean** - The statistical average of all broker estimates determined to be on the majority accounting basis. Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.

--> this is a analyst forecast variable

**TR.EPSActSurprise** - The difference between the actual and the last mean of the period, expressed as a percentage. Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.

--> forecast error between actual EPS and TR.EPSMean

<img src="../Pictures/Screenshot 2023-02-15 185222.png" width=500 height=300 />

## Part 2: Factors

### Firm charateristics

1. **TR.RevenueActValue** - The company's actual value normalized to reflect the I/B/E/S default currency and corporate actions (e.g. stock splits). Revenue (or Sales) is a corporation's net revenue, generally derived from core business activities. For non-financial companies, the calculation of net revenue (or net turnover) in most markets generally involves subtracting transportation and related operational costs from gross revenue/sales. Revenue recognition practices vary significantly from market to market, though generally the recording of revenue is based upon sales invoices issued (or anticipated for forecast purposes) during the accounting period.

2. **TR.F.EV** - Enterprise Value is the Company's total economic value OR the theoretical takeover price, often used as an alternative to market capitalization. Enterprise Value represents Market Capitalization TR.F.MktCap + Debt including Preferred Equity & Minority Interest - Total [TR.F.DebtlnclPrefEqMinlntrTot] (-) Cash & Short Term Investments - Total [TR.F.CashSTlnvstTot]. Market Capitalization [TR.F.MktCap] is required. The data item is calculated for all periodicities. It is applicable to all industries.

3. **TR.LTGMean** - Long Term Growth - Mean:  The statistical average of all broker estimates determined to be on the majority accounting basis. Long-term growth is an estimate of the compound average rate of EPS growth an analyst expects over a period of three to five ears.

### Analysts
1. **TR.NumberOfAnalysts** - Number of sell-side analysts covering the security.

2. **TR.RecMean** - Recommendation Numeric Mean based on the Standard Scale Of Strong Buy (1 ), Buy (2), Hold (3), Sell (4), and Strong Sell (5).

### Macro characteristics

1. **TR.WACCInflationAdjRiskFreeRate** -  Inflation adjusted risk free rate used is Of the primary index for the domicile country Of the company. Inflation adjusted risk free rate (Rf) is calculated from the US 10-year treasury yield plus the difference between in the 10-year forecasted inflation rate between the given country and the US.

### Loss Firm Status (0 or 1)

1. **TR.F.NetIncAfterTax** Net Income after Tax [SIAT] represents the income/expense after all operating and non-operating income and expense, reserves, income taxes, but before equity in earnings, minority interest, extraordinary items, after-tax adjustments, discontinued operations and preferred dividends. Applicable to all Industries. Net Income after Tax [SIAT] includes: • Net Income after Tax [XIAT]

### More difficult Factors (yearly):

#### ESG

1. **TR.TRESGScore** Refinitiv ESG Score is an overall company score based on the self-reported information in the environmental, social and corporate governance pillars.

#### Proportion of independent Directors

1. **TR.AnalyticIndepBoard** - Independent Board Members - Percentage of independent board members as reported by the company.

2. **TR.GovernancePillarScore** - Governance Pillar Score - The corporate governance pillar measures a company's systems and processes, which ensure that its board members and executives act in the best interests of its long term shareholders. It reflects a company's capacity, through its use of best management practices, to direct and control its rights and responsibilities through the creation of incentives, as well as checks and balances in order to generate long term shareholder value.

## Part 3: Regression

### continue..

In [29]:
economic_story = ['TR.EPSactValue.date', 'TR.EPSActValue', "TR.EPSMean", "TR.EPSActSurprise", 'TR.RevenueActValue', "TR.F.EV", "TR.LTGMean",  "TR.NumberOfAnalysts", "TR.RecMean", "TR.WACCInflationAdjRiskFreeRate", "TR.F.NetIncAfterTax"]
# , "TR.AdjSharesHeld"

In [30]:
df, e = ek.get_data('0#.SPX', economic_story, parameters = {'SDate':'-1','EDate':'-5','Period':'FQ0','Frq':'FQ'})
df["Date"] = pd.to_datetime(df["Date"])
df

C:\Users\alexa\mambaforge\envs\AnalystForecast\Lib\site-packages\pandas\core\dtypes\cast.py:1134: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
C:\Users\alexa\mambaforge\envs\AnalystForecast\Lib\site-packages\pandas\core\dtypes\cast.py:1153: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Long Term Growth - Mean,Number of Analysts,Recommendation - Mean (1-5),"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax
0,POOL.OQ,2022-10-20 07:00:00+00:00,4.78,4.5875,4.196,1615339000.0,14061600372.299999,<NA>,10,2.27273,3.747421,189965000.0
1,POOL.OQ,2022-07-21 07:00:00+00:00,7.63,7.517,1.503,2055818000.0,15561870705.290001,<NA>,9,2.2,3.092855,307205000.0
2,POOL.OQ,2022-04-21 07:00:00+00:00,4.23,3.14867,34.342,1412650000.0,18436823034.200001,<NA>,9,2.0,2.325202,179203000.0
3,POOL.OQ,2022-02-17 07:00:00+00:00,2.63,1.875,40.267,1035557000.0,23848820586.0,<NA>,8,2.3,1.515266,107542000.0
4,POOL.OQ,2021-10-21 07:00:00+00:00,4.51,3.84833,17.194,1411448000.0,17699543822.66,<NA>,8,2.3,1.527139,184573000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2510,AVY.N,2022-10-26 06:45:00+00:00,2.46,2.46323,-0.131,2317100000.0,16225017418.9,7.526,13,2.14286,3.747421,221500000.0
2511,AVY.N,2022-07-27 06:45:00+00:00,2.64,2.36485,11.635,2347000000.0,16504297189.84,6.94967,13,2.0,3.092855,214500000.0
2512,AVY.N,2022-04-26 06:45:00+00:00,2.4,2.17008,10.595,2349300000.0,17537077488.32,7.76667,12,2.06667,2.325202,198200000.0
2513,AVY.N,2022-02-02 06:45:00+00:00,2.13,2.12375,0.294,2183200000.0,20873039410.310001,9.16,12,2.14286,1.515266,183100000.0


In [31]:
4.78/4.5875

1.0419618528610355

In [32]:
economic_story_yearly = ["TR.TRESGScore.date", "TR.TRESGScore", "TR.GovernancePillarScore", "TR.AnalyticIndepBoard"]

In [33]:
df2, e = ek.get_data('0#.SPX', economic_story_yearly, parameters = {'SDate':'0','EDate':'-10','Period':'FY0','Frq':'FY'})
df2

,Instrument,Date,ESG Score,Governance Pillar Score,Independent Board Members
0,POOL.OQ,2021-12-31T00:00:00Z,48.718747,79.856229,80.0
1,POOL.OQ,2020-12-31T00:00:00Z,40.485976,69.05684,88.888889
2,POOL.OQ,2019-12-31T00:00:00Z,44.618552,80.983486,88.888889
3,POOL.OQ,2018-12-31T00:00:00Z,42.83681,66.013223,87.5
4,POOL.OQ,2017-12-31T00:00:00Z,39.816484,54.70234,75.0
...,...,...,...,...,...
5505,AVY.N,2016-01-02T00:00:00Z,65.011968,56.668962,90.0
5506,AVY.N,2015-01-03T00:00:00Z,70.474405,79.199798,91.666667
5507,AVY.N,2013-12-28T00:00:00Z,67.491583,69.926652,84.615385
5508,AVY.N,2012-12-29T00:00:00Z,73.981251,74.645174,83.333333


In [39]:
df.dtypes

Instrument                                                  string
Date                                           datetime64[ns, UTC]
Earnings Per Share - Actual                                Float64
Earnings Per Share - Mean                                  Float64
Earnings Per Share - Actual Surprise                       Float64
Revenue - Actual                                           Float64
Enterprise Value                                           Float64
Long Term Growth - Mean                                    Float64
Number of Analysts                                           Int64
Recommendation - Mean (1-5)                                Float64
WACC Inflation Adjusted Risk Free Rate, (%)                Float64
Net Income after Tax                                       Float64
dtype: object

In [37]:
df.value_counts("")

Number of Analysts
16    177
17    170
15    159
12    151
18    148
14    138
11    131
13    130
19    125
21    123
22    115
20    107
10     90
23     84
8      78
9      73
24     73
25     71
26     54
7      52
27     46
28     37
6      27
30     22
29     22
5      16
4      11
3       9
31      9
32      9
34      8
38      5
40      5
42      4
33      4
36      4
39      3
0       3
37      3
1       3
43      3
41      2
2       2
35      1
dtype: int64

In [38]:
df.value_counts("Date")

Date
2021-10-28 07:00:00+00:00    10
2022-07-28 07:00:00+00:00     9
2022-04-28 16:05:00+00:00     8
2022-07-27 07:00:00+00:00     8
2021-10-27 07:00:00+00:00     8
                             ..
2022-02-08 07:01:00+00:00     1
2022-02-08 07:00:00+00:00     1
2022-02-08 06:50:00+00:00     1
2022-02-08 06:40:00+00:00     1
2022-11-30 16:05:00+00:00     1
Length: 1748, dtype: int64